In [1]:
import os
import pandas as pd
import gradio as gr

# LangChain Imports
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

C:\Users\ketan\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from dotenv import load_dotenv
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [4]:
# ---------------- CONFIGURATION ----------------
INPUT_CSV = "places_final_ai_ready.csv"  # Your specific file
EMBEDDING_MODEL = "text-embedding-3-small"
LLM_MODEL = "gpt-3.5-turbo"

# Check for API Key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY environment variable is not set. Please set it before running.")

# ---------------- 1. INITIALIZE VECTOR DATABASE ----------------
def initialize_knowledge_base():
    print(f"Loading data from '{INPUT_CSV}'...")
    
    if not os.path.exists(INPUT_CSV):
        raise FileNotFoundError(f"File not found: {INPUT_CSV}")

    # Load the CSV file
    # source_column is optional, but helps if you want to know which row an answer came from
    loader = CSVLoader(file_path=INPUT_CSV, encoding="utf-8")
    documents = loader.load()
    
    print(f"Loaded {len(documents)} rows/documents.")

    print("Creating FAISS Vector Index (Embedding data)...")
    embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
    
    # Create the Vector Store from documents
    vectorstore = FAISS.from_documents(documents, embeddings)
    
    print("FAISS Index created successfully.")
    return vectorstore

# Initialize the DB once when the script starts
vectorstore = initialize_knowledge_base()

# ---------------- 2. SETUP CONVERSATION CHAIN ----------------
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI(temperature=0, model_name=LLM_MODEL)
    
    memory = ConversationBufferMemory(
        memory_key="chat_history", 
        return_messages=True
    )
    
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

# Create the chain
qa_chain = get_conversation_chain(vectorstore)

# ---------------- 3. GRADIO INTERFACE FUNCTION ----------------
def chat_function(message, history):
    """
    Logic for the chatbot. 
    'message' is the user's input.
    'history' is the chat history passed by Gradio (not used here as LangChain handles its own memory).
    """
    response = qa_chain.invoke({"question": message})
    return response["answer"]

# ---------------- 4. LAUNCH UI ----------------
if __name__ == "__main__":
    print("Starting Gradio Server...")
    
    # Create the Chat Interface
    demo = gr.ChatInterface(
        fn=chat_function,
        title="Places AI Chatbot",
        description=f"Ask questions about data in {INPUT_CSV}",
        theme="soft"
    )
    
    # Launch
    demo.launch()

Loading data from 'places_final_ai_ready.csv'...
Loaded 133 rows/documents.
Creating FAISS Vector Index (Embedding data)...
FAISS Index created successfully.
Starting Gradio Server...


C:\Users\ketan\anaconda3\Lib\site-packages\gradio\chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
